## Model setup

In [1]:
# load required packages 
import itertools
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

import ixmp as ix
import message_ix

from message_ix.utils import make_df

os.chdir('H:\\message_trade\\analysis\\1_toy_scenario')

In [2]:
# launch the IX modeling platform using the local default database                         
mp = ix.Platform(dbtype='HSQLDB')

INFO:root:launching ixmp.Platform with local HSQLDB database at 'C:\Users\shepard\.local\ixmp\localdb\default'


In [3]:
model = "Toy Global Model"
scen = "baseline"
annot = "Develop a toy model to illustrate global trade incorporation" 

scenario = message_ix.Scenario(mp, model, scen, version='new', annotation=annot)

## Time and space sets

In [4]:
# Time: Add horizon
horizon = range(2010, 2031, 5)
scenario.add_horizon({'year': horizon})

In [5]:
# Spatial: Add spatial set
regions = ["ASIA", "OECD", "REF"]
scenario.add_spatial_sets({'regions' : regions})
scenario.set('lvl_spatial')

0      World
1     global
2    regions
dtype: object

In [6]:
# Add required sets: commodity, level, mode
scenario.add_set("commodity", ["electricity", "light", "other_electricity"])
scenario.add_set("level", ["secondary", "final", "useful"])
scenario.add_set("mode", "standard")

In [7]:
# Add technology set
plants = [
    "coal_ppl", 
    "gas_ppl", 
    "oil_ppl", 
    "bio_ppl", 
    "hydro_ppl",
    "wind_ppl", 
    "solar_pv_ppl", # actually primary -> final
]
secondary_energy_techs = plants + ['import']

final_energy_techs = ['electricity_grid']

lights = [
    "bulb", 
    "cfl", 
]
useful_energy_techs = lights + ['appliances']
technologies = secondary_energy_techs + final_energy_techs + useful_energy_techs
scenario.add_set("technology", technologies)

## Parameters

### General parameters

In [8]:
# Add "general" parameters: interest rate
rate = [0.05] * len(horizon)
unit = ['%'] * len(horizon)
scenario.add_par("interestrate", key=horizon, val=rate, unit=unit)

### Commodity parameters (demand)

In [10]:
# Add "demand" parameters: demand
gdp = pd.read_csv('data\\gdp.csv')
gdp['demand'] = gdp.gdp * 0.7

In [12]:
# Add commodity demands: electricity (useful), other electricity (useful), light
base = pd.read_csv('data\\base.csv')

demand_base = base
demand_base['time'] = 'year'
demand_base['unit'] = 'GWa'

# Other electricity
demand_per_year = 55209. / 8760 # from IEA statistics

elec_demand = demand_base
elec_demand['commodity'] = 'other_electricity'
elec_demand['level'] = 'useful'
elec_demand['value'] = gdp.demand * demand_per_year

# Light
demand_per_year = 6134. / 8760 # from IEA statistics

light_demand = demand_base
light_demand['commodity'] = 'light'
light_demand['level'] = 'useful'
light_demand['value'] = gdp.demand * demand_per_year

scenario.add_par("demand", elec_demand)
scenario.add_par("demand", light_demand)

### Technology parameters

#### Set up input/output base dataframes

In [13]:
# Add vintage and active years
year_df = scenario.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']

In [14]:
# Combinations of nodes
base_comb = pd.DataFrame(data = list(itertools.product(base['node'], base['node'])),
                         columns = ['node1', 'node2'])
base_comb = base_comb.drop_duplicates()

# Add vintage and active years to each combination
base_comb = pd.concat([base_comb]*len(vintage_years), ignore_index=True)
base_comb = base_comb.sort_values(by = ['node1', 'node2'])
base_comb = base_comb.reset_index()
base_comb = base_comb.drop(columns = 'index')

# Repeat years for each combination of nodes
year_df2 = year_df.append(year_df, ignore_index = True)
nt = len(regions)*len(regions)-2

for i in range(nt):
    year_df2 = year_df2.append(year_df, ignore_index = True)
    
base_comb['year_vtg'] = year_df2['year_vtg']
base_comb['year_act'] = year_df2['year_act']

In [15]:
# Set up base input dataframe
base_input = base_comb
base_input['mode'] = 'standard'
base_input['commodity'] = 'electricity'
base_input['time'] = 'year'
base_input['time_origin'] = 'year'
base_input = base_input.rename(index = str, 
                               columns = {'node1':'node_origin', 'node2':'node_loc'})

# Set up base output dataframe
base_output = base_input
base_output = base_output.rename(index = str,
                                 columns = {'node_origin':'node_loc', 'node_loc':'node_dest', 'time_origin':'time_dest'})

#### Set up input parameters

In [16]:
grid = pd.DataFrame(dict(
        technology = 'electricity_grid',
        level = 'secondary',
        value = 1.0,
        unit = '%',
        **base_input
        ))

bulb = pd.DataFrame(dict(
        technology = 'bulb',
        level = 'final',
        value = 1.0,
        unit = '%',
        **base_input
        ))

cfl = pd.DataFrame(dict(
        technology = 'cfl',
        level = 'final',
        value = 0.3, 
        unit = '%',
        **base_input
        ))

app = pd.DataFrame(dict(
        technology = 'appliances',
        level = 'final',
        value = 1.0,
        unit = '%',
        **base_input
        ))

scenario.add_par("input", grid)
scenario.add_par("input", bulb)
scenario.add_par("input", cfl)
scenario.add_par("input", app)

#### Set up output parameters

In [17]:
imports = make_df(base_output, technology='import', commodity='electricity', 
                  level='secondary', value=1., unit='%')
scenario.add_par('output', imports)

grid = make_df(base_output, technology='electricity_grid', commodity='electricity', 
               level='final', value=0.873, unit='%')
scenario.add_par('output', grid)

bulb = make_df(base_output, technology='bulb', commodity='light', 
               level='useful', value=1., unit='%')
scenario.add_par('output', bulb)

cfl = make_df(base_output, technology='cfl', commodity='light', 
              level='useful', value=1., unit='%')
scenario.add_par('output', cfl)

app = make_df(base_output, technology='appliances', commodity='other_electricity', 
              level='useful', value=1., unit='%')
scenario.add_par('output', app)

coal = make_df(base_output, technology='coal_ppl', commodity='electricity', 
               level='secondary', value=1., unit='%')
scenario.add_par('output', coal)

gas = make_df(base_output, technology='gas_ppl', commodity='electricity', 
              level='secondary', value=1., unit='%')
scenario.add_par('output', gas)

oil = make_df(base_output, technology='oil_ppl', commodity='electricity', 
              level='secondary', value=1., unit='%')
scenario.add_par('output', oil)

bio = make_df(base_output, technology='bio_ppl', commodity='electricity', 
              level='secondary', value=1., unit='%')
scenario.add_par('output', bio)

hydro = make_df(base_output, technology='hydro_ppl', commodity='electricity', 
                level='secondary', value=1., unit='%')
scenario.add_par('output', hydro)

wind = make_df(base_output, technology='wind_ppl', commodity='electricity', 
               level='secondary', value=1., unit='%')
scenario.add_par('output', wind)

solar_pv = make_df(base_output, technology='solar_pv_ppl', commodity='electricity', 
                   level='final', value=1., unit='%')
scenario.add_par('output', solar_pv)

#### Set up technical lifetimes

In [18]:
base_technical_lifetime = base_input[['node_loc', 'year_vtg']]
base_technical_lifetime = base_technical_lifetime.drop_duplicates()
base_technical_lifetime['unit'] = 'y'

# Assume same lifetime for all region (CHANGE)
lifetimes = {
    'coal_ppl': 40,
    'gas_ppl': 30,
    'oil_ppl': 30,
    'bio_ppl': 30,
    'hydro_ppl': 60,
    'wind_ppl': 20,
    'solar_pv_ppl': 20,
    'bulb': 1,
    'cfl': 10,
}

for t, v in lifetimes.items():
    df = make_df(base_technical_lifetime, technology = t, value = v)
    scenario.add_par('technical_lifetime', df)
    

#### Set up capacity factor

In [19]:
base_cf = base_input[['node_loc', 'year_vtg', 'year_act']]
base_cf = base_cf.drop_duplicates()
base_cf['time'] = 'year'
base_cf['unit'] = '%'

capacity_factor = {
    'coal_ppl': 0.85,
    'gas_ppl': 0.75,
    'oil_ppl': 0.75,
    'bio_ppl': 0.75,
    'hydro_ppl': 0.5,
    'wind_ppl': 0.2,
    'solar_pv_ppl': 0.15,
    'bulb': 0.1, 
    'cfl':  0.1, 
}

for t, v in capacity_factor.items():
    df = make_df(base_cf, technology = t, value = v)
    scenario.add_par('capacity_factor', df)

### Technoeconomic (cost) parameters

In [20]:
# Investment costs ($/GWa)
base_ic = base_input[['node_loc', 'year_vtg']]
base_ic = base_ic.drop_duplicates()
base_ic['unit'] = 'USD/GWa'

# in $ / kWa
costs = {
    'coal_ppl': 1500,
    'gas_ppl':  870,
    'oil_ppl':  950,
    'hydro_ppl': 3000,
    'bio_ppl':  1600,
    'wind_ppl': 1100,
    'solar_pv_ppl': 4000,
    'bulb': 5,
    'cfl':  900, 
}

for t, v in costs.items():
    df = make_df(base_ic, technology=t, value=v * 1e6)
    scenario.add_par('inv_cost', df)

In [21]:
# Fixed costs ($/GWa)
base_fc = base_input[['node_loc', 'year_vtg', 'year_act']]
base_fc = base_fc.drop_duplicates()
base_fc['unit'] = 'USD/GWa'

# in $ / kWa
costs = {
    'coal_ppl': 40,
    'gas_ppl':  25,
    'oil_ppl':  25,
    'hydro_ppl': 60,
    'bio_ppl':  30,
    'wind_ppl': 40,
    'solar_pv_ppl': 25,
}

for t, v in costs.items():
    df = make_df(base_fc, technology=t, value=v * 1e6)
    scenario.add_par('fix_cost', df)

In [22]:
# Fixed costs ($/GWa)
base_vc = base_input[['node_loc', 'year_vtg', 'year_act']]
base_vc = base_vc.drop_duplicates()
base_vc['unit'] = 'USD/GWa'
base_vc['mode'] = 'standard'
base_vc['time'] = 'year'

# in $ / kWa
costs = {
    'coal_ppl': 40,
    'gas_ppl':  25,
    'oil_ppl':  25,
    'hydro_ppl': 60,
    'bio_ppl':  30,
    'wind_ppl': 40,
    'solar_pv_ppl': 25,
}

for t, v in costs.items():
    df = make_df(base_vc, technology=t, value=v * 1e6)
    scenario.add_par('var_cost', df)

### Dynamic behavior parameters

In [23]:
# Growth activity parameter
growthdf = pd.read_csv('data\\growth_activities.csv')
growthdf = growthdf[['node_loc', 'year_act', 'technology', 'value', 'time', 'unit']]

growth_technologies = growthdf.technology
growth_technologies = growth_technologies.drop_duplicates()

for t in growth_technologies:
    df = growthdf[growthdf.technology == t]
    scenario.add_par('growth_activity_up', df)
    

In [24]:
# Initial activity parameter (final demand activity)
base_initial = base_input[['node_loc', 'year_act', 'time']]
base_initial = base_initial.drop_duplicates()
base_initial['unit'] = '%'
base_initial = base_initial[base_initial.year_act.isin(horizon[1:])]

light_demand_active = light_demand[light_demand.year.isin(horizon[1:])]
light_demand_active = light_demand_active[['node', 'year', 'value']]

for t in lights:
    df = pd.merge(base_initial, light_demand_active, left_on = ['year_act', 'node_loc'], right_on = ['year', 'node'])
    df['technology'] = t
    df['value'] = df.value * 0.01
    scenario.add_par('initial_activity_up', df)

In [25]:
# Activity bounds (historic), lower and upper
activitydf = pd.read_csv('data\\base_activity.csv')

active_tech = ['coal_ppl', 'gas_ppl', 'oil_ppl', 'hydro_ppl',
               'bio_ppl', 'wind_ppl', 'solar_pv_ppl', 'cfl']

for t in active_tech:
    df = activitydf[['node_loc', 'year_act', 'mode', 'time', 'unit', t]]
    df = df.rename(index = str, columns = {t:'value'})
    df.insert(1, 'technology', t)
    scenario.add_par('bound_activity_up', df)
    scenario.add_par('bound_activity_lo', df)

In [26]:
# Base capacity bounds
base_capacity = activitydf[['node_loc']]
base_capacity = base_capacity.drop_duplicates()
base_capacity['unit'] = 'GWa'
base_capacity['year_vtg'] = 2010
base_capacity['unit'] = 'GWa'

for t in active_tech:
    act = activitydf[t]
    cf = capacity_factor[t]
    cap = act/8760/cf
    
    df = make_df(base_capacity, value = cap, technology = t)
    scenario.add_par('bound_new_capacity_up', df)

## Commit and solve

In [27]:
# Commit
comment = 'initial commit for global toy model'
scenario.commit(comment)
scenario.set_as_default()

In [28]:
# Solve
scenario.solve()
scenario.var('OBJ')['lvl']

CalledProcessError: Command 'gams "C:\ProgramData\Anaconda3\lib\site-packages\message_ix\model\MESSAGE_run.gms" --in="C:\ProgramData\Anaconda3\lib\site-packages\message_ix\model\data\MsgData_Toy_Global_Model_baseline.gdx" --out="C:\ProgramData\Anaconda3\lib\site-packages\message_ix\model\output\MsgOutput_Toy_Global_Model_baseline.gdx" --iter="C:\ProgramData\Anaconda3\lib\site-packages\message_ix\model\output\MsgIterationReport_Toy_Global_Model_baseline.gdx" LogOption=4' returned non-zero exit status 3.